In [17]:
from neo4j import GraphDatabase
import pandas as pd
import numpy as np
from py2neo import Graph, Node, Relationship

In [26]:
uri = 'bolt://3.239.243.19:7687'
pwd = 'chances-jelly-adjective'

In [20]:
graph = Graph(url, auth=('neo4j', pwd))


In [23]:
query = """
    MATCH (p1:Person)-[:DEFENDER_COMMANDER]->(b:Battle)<-[:ATTACKER_COMMANDER]-(p2:Person)
    RETURN p2.name AS attacker, b.name AS battle, p1.name AS defender LIMIT 10
"""

result_df = graph.query(query).to_data_frame()
result_df.head(10)

,attacker,battle,defender
0,Jaime Lannister,Battle of the Golden Tooth,Vance
1,Jaime Lannister,Battle of the Golden Tooth,Clement Piper
2,Gregor Clegane,Battle at the Mummer's Ford,Beric Dondarrion
3,Andros Brax,Battle of Riverrun,Tytos Blackwood
4,Jaime Lannister,Battle of Riverrun,Tytos Blackwood
5,Andros Brax,Battle of Riverrun,Edmure Tully
6,Jaime Lannister,Battle of Riverrun,Edmure Tully
7,Harrion Karstark,Battle of the Green Fork,Tywin Lannister
8,Medger Cerwyn,Battle of the Green Fork,Tywin Lannister
9,Roose Bolton,Battle of the Green Fork,Tywin Lannister


In [27]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [28]:
conn = Neo4jConnection(uri=uri, user='neo4j', pwd=pwd)


In [29]:
result = conn.query('MATCH (n) RETURN COUNT(n) AS ct')
print(result)

[<Record ct=2644>]
